# Task 2

In [1]:
%matplotlib nbagg

import numpy as np
import strid
import matplotlib.pyplot as plt
import scipy.signal
import covSSI

/Users/emilneby/miniforge3/envs/tkt4550/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Perform covSSI on generated data to get the estimated modes
stochastic_data_path = "results/data-stochastic_8_floor.npz"
combined_data_path = "results/data-combined_8_floor.npz"
ssid, modes = covSSI.covSSI(stochastic_data_path)

### Task 2.1 - finding relative difference

In [3]:
cluster_meat = []
num_modes = 0

for order in range(49, 5, -1):
    modes_of_order = modes[order]
    #print("Order: " + str(order))
    num_modes += len(modes_of_order)
    counter = 0
    for mode in modes_of_order:
        counter += 1
        neighbour = strid.find_nearest_neighbour(mode, modes[order-1])

        if (np.isnan(strid.rel_diff_freq(neighbour.f, mode.f))):
            mode.delta_frequency = 1
        else:
            mode.delta_frequency = strid.rel_diff_freq(neighbour.f, mode.f)

        if (np.isnan(np.abs(neighbour.xi - mode.xi))):
            mode.delta_damping = 1
        else:
            mode.delta_damping = np.abs(neighbour.xi - mode.xi)

        if (np.isnan(strid.utils.modal_assurance_criterion(neighbour.v, mode.v))):
            mode.mac = 1
        else:
            mode.delta_mac = strid.utils.modal_assurance_criterion(neighbour.v, mode.v)

        cluster_meat.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])

cluster_meat = np.array(cluster_meat)
# !!! Have to handle modes in the lowest order!!!

/Users/emilneby/PyCharmProjects/TKT4550_Prosjekt/strid/utils.py:729: RuntimeWarning: invalid value encountered in double_scalars
  return self.eigenvalue.imag / np.sqrt(1-self.xi**2)


In [4]:
print(np.max(cluster_meat))

1.0


### Task 2.2 -  Using K-means clustering to separate all the poles into two groups

In [5]:
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0).fit(cluster_meat)
labels = kmeans.labels_

In [6]:
#Assign label to each mode
physical_coordinates = []
mathematical_coordinates = []
count = 0
physical_modes_dict = {}
physical_modes_list = []
num_modes = 0
for order in range(49, 5, -1):
    modes_of_order = modes[order]
    #print("Order: " + str(order))
    physical_modes_in_order = []
    for mode in modes_of_order:
        mode.physical = labels[count]
        if (mode.physical == 0):
            physical_modes_in_order.append(mode)
            physical_modes_list.append(mode)
            physical_coordinates.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])
        else:
            mathematical_coordinates.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])
        count += 1
    physical_modes_dict[order] = physical_modes_in_order
    num_modes += len(physical_modes_in_order)

physical_coordinates = np.array(physical_coordinates)
mathematical_coordinates = np.array(mathematical_coordinates)
physical_modes_list = np.array(physical_modes_list)


In [7]:
print(physical_modes_list.shape[0])

1090


In [8]:
%matplotlib nbagg
#Making a 3D scatterplot of frequency, damping and MAC number divided into physical and mathematical clusters
fig = plt.figure()
ax = plt.axes(projection='3d')

fg = ax.scatter3D(physical_coordinates[:,0], physical_coordinates[:,1], physical_coordinates[:,2], c="blue", label= "Physical modes")
ax.scatter3D(mathematical_coordinates[:,0], mathematical_coordinates[:,1], mathematical_coordinates[:,2], c="red", label="Mathematical modes")
ax.legend()
ax.set_xlabel("Frequency")
ax.set_ylabel("Damping")
ax.set_zlabel("MAC")

print("Number of physical modes = " + str(physical_coordinates.shape[0]))
print("Number of mathematical modes = " +str(mathematical_coordinates.shape[0]))

<IPython.core.display.Javascript object>

Number of physical modes = 1090
Number of mathematical modes = 120


# Task 3
### Task 3.1 Detecting structural modes by hierachical clustering
Using hierarchical clustering with a signle linkage method with a cut-off-distance of $d_c=0.04$
$$ d_{c_{i,j}} = d \lambda_{i,j} + (1- MAC_{i,j}) $$

In [9]:
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html
# Plotting dendogram: https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html
# https://www.youtube.com/watch?v=v7oLMvcxgFY&ab_channel=KindsonTheTechPro

import strid
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering


#Since we want to define our own distance between two poles, we must compute a distance matrix we can feed into the clustering algorithm.


In [10]:
distance = strid.utils.distance_matrix(physical_modes_list)

/Users/emilneby/PyCharmProjects/TKT4550_Prosjekt/strid/utils.py:867: ComplexWarning: Casting complex values to real discards the imaginary part
  dist_matrix[i, j] = (np.abs((eigen_i - eigen_j)) / np.max([eigen_i, eigen_j])) + (


Distance matrix computational time = 17.19790005683899sec


In [11]:
#Check the integrity of the distance matrix
print(np.max(distance))
print(np.mean(distance))
print(np.min(distance))

2.2038210000141465
0.9218423894237713
0.0


In [12]:
#Perform the actual clustering
model = AgglomerativeClustering(n_clusters=None, affinity='precomputed', linkage='average', distance_threshold=0.04)
#The agglomerative clustering algorithm only finds 1 non-structural mode with single linkage method. Works better so far with 'complete' or 'average'.

In [13]:
y_hc = model.fit_predict(distance)
model = model.fit(distance)
print((y_hc))
print(np.max(y_hc))

[ 34 215 123 ...  78 162  34]
238


In [14]:
hierarchy = y_hc.tolist()
print(len(y_hc))
print(type(hierarchy))
#print(hierarchy)

num_modes_in_hierarchy = np.zeros((np.max(y_hc) + 1, 3))
for i in range(0, len(num_modes_in_hierarchy)):
    num_modes_in_hierarchy[i, 0] = hierarchy.count(i)
    num_modes_in_hierarchy[i, 1] = hierarchy.count(i)
    num_modes_in_hierarchy[i, 2] = i

print((num_modes_in_hierarchy))

1090
<class 'list'>
[[  3.   3.   0.]
 [  3.   3.   1.]
 [ 18.  18.   2.]
 [ 18.  18.   3.]
 [  3.   3.   4.]
 [  2.   2.   5.]
 [  4.   4.   6.]
 [  2.   2.   7.]
 [  2.   2.   8.]
 [  3.   3.   9.]
 [  3.   3.  10.]
 [  3.   3.  11.]
 [  2.   2.  12.]
 [  2.   2.  13.]
 [  4.   4.  14.]
 [  3.   3.  15.]
 [  3.   3.  16.]
 [  2.   2.  17.]
 [  5.   5.  18.]
 [  2.   2.  19.]
 [  2.   2.  20.]
 [  2.   2.  21.]
 [ 37.  37.  22.]
 [  6.   6.  23.]
 [  5.   5.  24.]
 [  3.   3.  25.]
 [  6.   6.  26.]
 [  3.   3.  27.]
 [  3.   3.  28.]
 [  2.   2.  29.]
 [  2.   2.  30.]
 [  3.   3.  31.]
 [  2.   2.  32.]
 [  3.   3.  33.]
 [  3.   3.  34.]
 [  3.   3.  35.]
 [  3.   3.  36.]
 [ 37.  37.  37.]
 [  2.   2.  38.]
 [  2.   2.  39.]
 [  2.   2.  40.]
 [  3.   3.  41.]
 [ 35.  35.  42.]
 [  9.   9.  43.]
 [  2.   2.  44.]
 [  2.   2.  45.]
 [  1.   1.  46.]
 [  6.   6.  47.]
 [  2.   2.  48.]
 [  2.   2.  49.]
 [ 40.  40.  50.]
 [ 10.  10.  51.]
 [  2.   2.  52.]
 [  3.   3.  53.]
 [  6.  

In [15]:
structural_coordinates = []
non_structural_coordinates = []

for i in range(0, len(y_hc)):
    if y_hc[i] == 1:
        structural_coordinates.append(physical_coordinates[i,:])
    else:
        non_structural_coordinates.append(physical_coordinates[i,:])

structural_coordinates = np.array(structural_coordinates)
non_structural_coordinates = np.array(non_structural_coordinates)

In [16]:
#Assign hierarchy to each mode
count = 0
for order in range(49, 5, -1):
    modes_of_order = physical_modes_dict[order]
    for mode in modes_of_order:
        mode.cluster = hierarchy[count]
        count += 1

### Task 3.2 Need to visualize the hierarchical clusters

# Task 4
### Task 4.1 - Using k-means to divide into one cluster with "many modes" (structural modes) and one cluster with "few/scattered modes" (mathematcal modes).

In [17]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(num_modes_in_hierarchy[:,:2])
labels2 = kmeans.labels_

In [18]:
#print(labels2)
print("Number of clusters identified: " +str(len(labels2)))
#print(labels2)

Number of clusters identified: 239


In [19]:
structural_hierarchies = []

for i in range(0, len(labels2)):
    if(labels2[i] == 1):
        structural_hierarchies.append(i)

print(structural_hierarchies)
#need to create a dictionary with colors to the clusters

[22, 37, 42, 50, 67, 75, 78, 87, 102, 105, 150, 162, 184, 202, 212, 218]


In [20]:
#Assign hierarchy to each mode
structural_modes_dict = {}
for order in range(49, 5, -1):
    modes_of_order = physical_modes_dict[order]
    structural_modes_in_order = []
    for mode in modes_of_order:
        #print(mode.f)
        if mode.cluster in structural_hierarchies:
            structural_modes_in_order.append(mode)

    structural_modes_dict[order] = structural_modes_in_order

In [21]:
%matplotlib nbagg
#Making a new stabilization diagram with the physical modes
stabdiag = strid.StabilizationDiagram()
stabdiag.plot_clusters(structural_modes_dict)

f, psd = ssid.psdy(nperseg=2**10)

stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)

<IPython.core.display.Javascript object>

Length of color list: 302


### Task 4.2 - Extract modal features of each detected mode as the average of all he components' features within each hierarchical cluster.

In [22]:
import hierarchical_modes

clustered_modes = hierarchical_modes.HierarchicalModes(stabdiag)

modes_in_clusters = clustered_modes.clusters_dict(structural_modes_dict)

In [23]:
#Verfying numbers of clusters up against stabilization diagram
print(len(modes_in_clusters))

8


In [24]:
hierarchies = (list(modes_in_clusters.values()))

est_frequencies = np.zeros(len(hierarchies))
est_damping = np.zeros(len(hierarchies))
est_modal_shapes = np.zeros(len(hierarchies))

for hier in hierarchies:
    frequencies_in_hierarchy = []
    damping_in_hierarchy = []
    mode_shapes_in_hierarchy = []
    for element in hier:
        frequencies_in_hierarchy.append(element.f)
        damping_in_hierarchy.append(element.xi)
        mode_shapes_in_hierarchy.append(element.v)

    est_frequencies[hierarchies.index(hier)] = np.mean(np.array(frequencies_in_hierarchy))
    est_damping[hierarchies.index(hier)] = np.mean(np.array(damping_in_hierarchy))
    est_modal_shapes[hierarchies.index(hier)] = np.mean(np.array(mode_shapes_in_hierarchy))

est_frequencies = np.sort(est_frequencies)
est_damping = np.sort(est_damping)
#est_modal_shapes = np.sort(est_modal_shapes)

/var/folders/xg/ddkjb2x96w167hqnv_gjv_v40000gn/T/ipykernel_10824/3324002376.py:18: ComplexWarning: Casting complex values to real discards the imaginary part
  est_modal_shapes[hierarchies.index(hier)] = np.mean(np.array(mode_shapes_in_hierarchy))


In [25]:
print(est_frequencies)

[ 2.98399651  8.70352989 14.19855537 19.18864758 23.44041853 27.11694957
 29.71109259 31.33904943]


In [26]:
print(est_modal_shapes)

[-0.00222728 -0.00057827 -0.00258535 -0.0072765  -0.00130218  0.00079302
  0.00064068 -0.00855711]


### Task 4.3 - Verify against ground truth

In [27]:
data = np.load("results/data-stochastic_8_floor.npz")

true_f = data["true_frequencies"]
true_xi = data["true_damping"]
true_modeshapes = data["true_modeshapes"]

In [28]:
print(true_f)

[ 2.9369931   8.71096352 14.18829253 19.18245625 23.52338443 27.06325197
 29.68151292 31.28900555]


In [29]:
print(np.abs(true_f-est_frequencies))

[0.04700341 0.00743364 0.01026284 0.00619133 0.08296591 0.0536976
 0.02957967 0.05004388]


In [30]:
modes = np.arange(1, len(true_f)+1)

plt.figure()
plt.plot(modes, np.abs(true_f-est_frequencies), label = '$\epsilon_f$')
plt.plot(modes, np.abs(true_xi-est_damping),label = '$\epsilon_{\\xi}$')

plt.legend()
plt.xlabel('Mode')
plt.grid()
plt.show()


<IPython.core.display.Javascript object>

In [105]:
print(est_modal_shapes)

[-0.00427678 -0.00407224 -0.00264787  0.00040658  0.00095547  0.00102093
  0.00357064  0.00619683]
